# SHOR with multiple errors, states probabilities


In [1]:
#QISKIT
import qiskit.quantum_info as qi

from qiskit import Aer
from qiskit import execute, transpile, assemble
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit import QuantumCircuit, execute
from qiskit.extensions import UnitaryGate
from qiskit.quantum_info.operators import Operator
from qiskit.visualization import plot_histogram
from qiskit.visualization import array_to_latex
from qiskit.quantum_info import Statevector

from qiskit.providers.aer import QasmSimulator
from qiskit.providers.aer.library import SaveDensityMatrix

#PYTHON
import numpy as np
from numpy import linalg
from numpy import pi
from sympy import *
from sympy import factorint
from scipy.sparse import csr_matrix

import random
from random import sample
import pandas as pd
import math
from fractions import Fraction
import matplotlib.pyplot as plt
import pickle

#CHECK N AND y, a
def check(number):
    if number % 2 == 0:
        print("Even, 2 is a divisor")
        return
        
    if isprime(number) == True:
        print("Prime number")
        return
    
    factorization = factorint(number)
    for i in range(len(factorization.keys())):
        p = list(factorization)[i]
        q = list(factorization.values())[i]
        if isprime(p) and q != 1:
            print("Prime Power: %i^%i, %i is a divisor" % ( p, q, p))
            return
    
    y = random.randrange(2, number - 1)
    a = math.gcd(y, number)
    if a > 1:
        print("Divisor: a = %i" %(a))
        return 
        
    else:
        print("y = %i and N = %i are coprime - r well defined" %  (y, N))
        return 
    
#MODULAR EXPONENTIATION GATE
def mod_exp(n, y, N, power):
    dim = 2**n
    matrix_0 = np.zeros((dim, dim))
    
    #CREATE F(X)
    for i in range(N):
        j = (i*y)%N
        matrix_0[j][i]=1
             
    for ii in range(N, dim):
        matrix_0[ii][ii] = 1

    temp = matrix_0       
    #temp = csr_matrix(matrix_0)
    for iii in range(power):
        matrix = temp.dot(temp)
        #matrix = temp.multiply(temp)
        temp = matrix

    #CREATE GATE
    U = UnitaryGate(temp)
    U.name = "%i^%i mod %i" % ( y, 2**power, N)
    c_U = U.control()
    return  c_U

#ERROR ROUTINE
def errori(circuit, position, n):
    epsilon = 0.5                                              
    extracted = random.uniform(0, 1)
    position = random.randint(0, n)
    
    if (extracted < epsilon):  
        circuit.x(position)   
    else:
        circuit.z(position)
    return circuit 

#IQFT                          
def qft_dagger(n, circuit, case, position):
    err = 0

    for qubit in range(n//2):
        circuit.swap(qubit, n-qubit-1)
        
    for j in range(n):
        for m in range(j):
            circuit.cp(-np.pi/float(2**(j-m)), m, j)
            circuit.barrier()
            if err < case:
                errori(circuit, position, n)
                circuit.barrier()
                err = err +  1
        circuit.barrier()
        
        circuit.h(j)
        circuit.barrier()
        if err < case:
            errori(circuit, position, n)
            circuit.barrier()
            err = err +  1
        
    return circuit


In [ ]:
N = 27
r = 16
y = 8
case_max = 1
case_step = 1
repetitions = 1

#coprimes = [2,4,7,8,11,13]
n = math.ceil(math.log(N,2))
n_count = 2*n

#MATRICES FOR MODEXP
matrices = []
for q in range(2*n):
    matrices.append(mod_exp(n, y, N, q))   

#DATA
data = []
f = open("FINAL_prj.txt", "w")

#USEFUL Js
useful_js = []
for c in range(1, r):
    useful_js.append(math.ceil((c*2**(n_count))/r))
    useful_js.append(math.floor((c*2**(n_count))/r))
useful_js_nodup = [*set(useful_js)]

#MAIN
for case in range(0, case_max, case_step):
    js = []
    
    for rep in range(repetitions):
        print(case, rep)
        #y = coprimes[random.randint(0, 5)]
        position = random.randint(0, 2*n)
        
        #CIRCUIT
        control = QuantumRegister(2*n, 'r1')
        target  = QuantumRegister(n, 't1')
        classic = ClassicalRegister(2*n, 'c')
        circuit = QuantumCircuit(control, target, classic)
        circuit.h(range(2*n))
        circuit.x(2*n + n -1)
        for q in range(2*n):
            circuit.append(matrices[q], [q] + [i+2*n for i in range(n)])   
        circuit.barrier()
        qft_dagger(2*n, circuit, case, position)
        circuit.barrier()
        circuit.measure(range(2*n), range(2*n))        

        #SIMULATION
        aer_sim = Aer.get_backend('aer_simulator')        
        job = aer_sim.run(transpile(circuit, aer_sim), shots=1, memory=True)
        readings = job.result().get_memory()
        #result = aer_sim.run(transpile(circuit, aer_sim), shots=2048).result()
        #counts  = result.get_counts(circuit)

        #PHASES   
        j = int(readings[0], 2)
        js.append(j)
    
        '''   
        phase = int(readings[0],2)/(2**n_count)                     
        #CONTINUED FRACTIONS
        if phase != 0:
            frac = Fraction(phase).limit_denominator(N)
            r, k = frac.denominator, frac.numerator
            guesses = [gcd(y**(r//2)-1, N), gcd(y**(r//2)+1, N)]
            data.append([r, guesses])
        '''

print('js ->', js)
print('r, fact ->', data)
f.write('js' + '   ' +  str(js))
f.write('\n')
f.write('r, fact' + '   ' +  str(data))
f.close()
